다음은 좀 더 간단하게 CNN을 만들기 위한 나름의 팁을 알려드리고자 합니다.
일단, 아까와 똑같은 걸 실행하겠습니다.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline


mnist = input_data.read_data_sets("./mnist", one_hot=True)
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

print "x_train: ", x_train.shape
print "y_train: ", y_train.shape
print "x_test: ", x_test.shape
print "y_test: ", y_test.shape

# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 100
display_step = 200

# Network parameters
n_input = 784
n_classes = 10
dropout = 0.5

# Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # to keep dropout probability

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
x_train:  (55000, 784)
y_train:  (55000, 10)
x_test:  (10000, 784)
y_test:  (10000, 10)


weight, bias를 매번 만들어줘야 하는 게 번거롭고, 또 tf.nn. 패키지 내의 다양한 연산들을 사용하다 보면 코드가 복잡해지는 등의 문제점이 있습니다. 이러한 점을 보완해보고자 지난 수업 때 나온 ** tf.get_variable() **이라는 함수를 활용하여, 간편성을 살린 함수를 만들어봤습니다.

In [2]:
def conv2d_simple(x, filter_size, dim_in, dim_out, name, strides=1):
    with tf.variable_scope(name) as scope:
        if type(filter_size)==int:
            filter_height=filter_size
            filter_width=filter_size
        elif type(filter_size)==list:
            filter_height,filter_width=filter_size
        w = tf.get_variable('w', shape=[filter_height, filter_width, dim_in, dim_out],
                initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))
            
        b = tf.get_variable('b', shape=[dim_out])
        x = tf.nn.conv2d(x,w,strides=[1,strides,strides,1],padding='SAME')
        x = tf.nn.bias_add(x,b)
    print "After convolution: ", x.get_shape().as_list()
    return tf.nn.relu(x)

In [3]:
def fc_simple(x,dim_out,name,is_last=False):
    with tf.variable_scope(name) as scope:
        size_arr = x.get_shape().as_list()
        dim_in = np.prod(np.asarray(size_arr[1:]))
        x = tf.reshape(x, [-1,dim_in])
        dim_in = x.get_shape().as_list()[1]
        wd = tf.get_variable('wd', shape=[dim_in, dim_out],
                initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))
        bd = tf.get_variable('bd', shape=[dim_out])
        fc = tf.add(tf.matmul(x, wd),bd)
        if not is_last:
            fc = tf.nn.relu(fc)
            print "Fully connected:   ", fc.get_shape().as_list()
            return fc
        print "Output:            ", fc.get_shape().as_list()
        return fc

In [4]:
def maxpool2d(x, k=2):
    pool=tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')
    print "After max_pooling: ", pool.get_shape().as_list()
    return pool

In [5]:
def CNN(x, weights, biases, dropout):
    
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # 1st Convolution Layer
    conv1 = tf.nn.conv2d(x, weights['wc1'], strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.bias_add(x, biases['bc1'])
    conv1 = tf.nn.relu(conv1)
    # Max Pooling (down-sampling)
    pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    # 2nd Convolution Layer
    conv2 = tf.nn.conv2d(pool1, weights['wc2'], strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.bias_add(x, biases['bc2'])
    conv2 = tf.nn.relu(conv2)
    # Max Pooling (down-sampling)
    pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    # Fully connected layer
    fc1 = tf.reshape(pool2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [6]:
def CNN_simple(x, dropout):
    with tf.variable_scope('CNN_simple'):
        # Reshape input picture
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

        # 1st Convolution Layer
        conv1 = conv2d_simple(x, 5, 1, 32, 'conv1')
        # Max Pooling (down-sampling)
        max1 = maxpool2d(conv1, k=2)

        # 2nd Convolution Layer
        conv2 = conv2d_simple(max1, 5, 32, 64, 'conv2')
        # Max Pooling (down-sampling)
        max2 = maxpool2d(conv2, k=2)

        # Fully connected layer
#        out = fc_simple(max2, 1024, n_classes, 'fc')
        fc = fc_simple(max2, 1024, 'fc1')
        out = fc_simple(fc, n_classes, 'fc2',is_last=True)
        return out

In [7]:
# Construct model
pred = CNN_simple(x, keep_prob)
# Define loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
# Define optimizer
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)
# Find correct prediction
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
# Get accuracy
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# initialize variables
init = tf.initialize_all_variables()
# config options
gpu_options = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.1)
config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True,
                        gpu_options=gpu_options)
# apply config when starting session
sess = tf.InteractiveSession(config=config)
init.run()
step = 1
while step * batch_size < training_iters:
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    # Run optimization
    sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
    if step % display_step == 0:
    # Calculate batch loss and accuracy
        loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
        print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
    step += 1
print("Optimization Finished!")
    # Calculate accuracy for 256 mnist test images
print("Testing Accuracy:", \
    sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

After convolution:  [None, 28, 28, 32]
After max_pooling:  [None, 14, 14, 32]
After convolution:  [None, 14, 14, 64]
After max_pooling:  [None, 7, 7, 64]
Fully connected:    [None, 1024]
Output:             [None, 10]
Iter 20000, Minibatch Loss= 0.206446, Training Accuracy= 0.89000
Iter 40000, Minibatch Loss= 0.187721, Training Accuracy= 0.95000
Iter 60000, Minibatch Loss= 0.073944, Training Accuracy= 0.97000
Iter 80000, Minibatch Loss= 0.028082, Training Accuracy= 0.99000
Iter 100000, Minibatch Loss= 0.004417, Training Accuracy= 1.00000
Iter 120000, Minibatch Loss= 0.013792, Training Accuracy= 0.99000
Iter 140000, Minibatch Loss= 0.003636, Training Accuracy= 1.00000
Iter 160000, Minibatch Loss= 0.002261, Training Accuracy= 1.00000
Iter 180000, Minibatch Loss= 0.000461, Training Accuracy= 1.00000
Optimization Finished!
('Testing Accuracy:', 0.984375)


** Tensor visualization **

사용한 weight나 tensor가 어떤 값을 가지는지 어떻게 볼 수 있는지 확인해보겠습니다.

In [8]:
#for var in tf.all_variables():
for var in tf.trainable_variables():
    print var.op.name, " : ", var.get_shape()

CNN_simple/conv1/w  :  (5, 5, 1, 32)
CNN_simple/conv1/b  :  (32,)
CNN_simple/conv2/w  :  (5, 5, 32, 64)
CNN_simple/conv2/b  :  (64,)
CNN_simple/fc1/wd  :  (3136, 1024)
CNN_simple/fc1/bd  :  (1024,)
CNN_simple/fc2/wd  :  (1024, 10)
CNN_simple/fc2/bd  :  (10,)


In [9]:
for var in tf.trainable_variables():
    if var.name.startswith('CNN_simple/conv1'):
        print var.op.name, var.get_shape()

CNN_simple/conv1/w (5, 5, 1, 32)
CNN_simple/conv1/b (32,)
